In [45]:
from Algorithms.clarke_wright import ClarkeWright
from Algorithms.client import Client
import pandas as pd
from datetime import datetime, timedelta
import random
import names
import json
import numpy as np

# Generate historized client data

In [46]:
def generate_clients(start_date, end_date, min_slots, max_slots, locations, max_timeslot_picks, timeslot_usage, num_clients=1):
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start_date, date_format)
    end_date = datetime.strptime(end_date, date_format)
    date_range = (end_date - start_date).days
    
    timeslots = ['morning', 'evening']
    clients = []

    for _ in range(num_clients):
        name = names.get_full_name()  # Generating random full names
        location = random.choice(locations)
        
        while True:
            # Generate all possible availability slots within the entire date range
            possible_slots = []
            for day in (start_date + timedelta(days=i) for i in range(date_range + 1)):
                for timeslot in timeslots:
                    slot = f"{day.strftime(date_format)}_{timeslot}"
                    if timeslot_usage.get(slot, 0) < max_timeslot_picks:
                        possible_slots.append(slot)

            if len(possible_slots) < min_slots:
                raise ValueError(f"Unable to generate the minimum number of slots ({min_slots}) for client {name}.")
            
            random.shuffle(possible_slots)
            availability = possible_slots[:max_slots]

            # Generate appointment day that is before or on the same day as the latest timeslot in availability
            latest_slot_day = max(datetime.strptime(slot.split('_')[0], date_format) for slot in availability)
            appointment_day = start_date + timedelta(days=random.randint(0, (latest_slot_day - start_date).days))
            appointment_hour = random.randint(0, 23)
            appointment_minute = random.randint(0, 59)
            appointment_time = datetime.combine(appointment_day, datetime.min.time()) + timedelta(hours=appointment_hour, minutes=appointment_minute)
            appointment_time_str = appointment_time.strftime(f"{date_format} %H:%M:%S")

            for slot in availability:
                timeslot_usage[slot] = timeslot_usage.get(slot, 0) + 1
            
            if len(availability) >= min_slots:
                break

        client = Client(name, location, availability, appointment_time_str)
        clients.append(client)
    
    return clients

# test online case

In [47]:
def get_definitive_timeslot_clarke(client, scheduled_definitive_appointments, distance_matrix_path):
    old_routes = []
    new_routes = []
    slots = [slot for slot in client.availability if slot in scheduled_definitive_appointments]
    
    # If there are no appointments scheduled yet in the availability slots just schedule the client
    if not slots:
        slot = client.availability[0]
        clarkewright = ClarkeWright([client])
        clarkewright.solve(slot, distance_matrix_path)
        scheduled_definitive_appointments[slot] = [[client], clarkewright.get_solution()]
        return f"Client {client.name} has been scheduled for {slot}", scheduled_definitive_appointments
    
    # If there are appointments scheduled in the availability slots, try to add the client to the existing route with the minimal cost
    for slot in slots:
        old_routes.append([scheduled_definitive_appointments[slot], slot])
        clients_new = scheduled_definitive_appointments[slot][0] + [client]
        clarkewright_new = ClarkeWright(clients_new)
        clarkewright_new.solve(slot, distance_matrix_path)
        new_routes.append([[clients_new, clarkewright_new.get_solution()], slot])

    # Compare the costs of all routes and pick the route with the smallest delta
    min_delta = float('inf')
    best_route = None
    for old_route, new_route in zip(old_routes, new_routes):
        old_cost = old_route[0][1][1]  # Extracting the cost from the old route
        new_cost = new_route[0][1][1]  # Extracting the cost from the new route
        delta = new_cost - old_cost
        if delta < min_delta:
            min_delta = delta
            best_route = new_route
    
    # Update the scheduled_definitive_appointments with the best route
    if best_route:
        slot = best_route[1]
        scheduled_definitive_appointments[slot] = [best_route[0][0], best_route[0][1]]
        return f"Client {client.name} has been scheduled for {slot} with the minimal delta cost.", scheduled_definitive_appointments

In [48]:

config = {
    'num_clients': 25,
    'start_date': "2024-06-01",
    'end_date': "2024-06-14",
    'min_slots': 2,
    'max_slots': 2,
    'locations': ['Asten Heusden Ommel', 'Deurne Vlierden', 'Geldrop', 'Gemert Handel', 'Helmond',
                  'Helmond Brandevoort', 'Mierlo', 'Nuenen Gerwen Nederwetten', 'Someren'],
    'distance_matrix_path': "..//Data//distance_matrix.csv",
    'batch_size': 7,  # For offline use case, processing batches of clients every 7 days
    'max_timeslot_picks': 3  # Maximum number of timeslots a single slot can be picked
}

# random seed
random.seed(42)

In [49]:
def online_use_case(config):
    scheduled_definitive_appointments = {}
    timeslot_usage = {}
    
    for _ in range(config['num_clients']):
        clients = generate_clients(config['start_date'], config['end_date'], config['min_slots'], config['max_slots'], config['locations'], config['max_timeslot_picks'], timeslot_usage, num_clients=1)
        client = clients[0]
        print(f"names: {client.name}, location: {client.location}, availability: {client.availability}, appointment_time: {client.appointment_time}")
        message, scheduled_definitive_appointments = get_definitive_timeslot_clarke(client, scheduled_definitive_appointments, config['distance_matrix_path'])
        print(message)
        
        print(scheduled_definitive_appointments)

In [50]:
print("Online Use Case Results:")
online_use_case(config)

Online Use Case Results:
names: James Lawrence, location: Gemert Handel, availability: ['2024-06-04_evening', '2024-06-03_evening'], appointment_time: 2024-06-04 10:17:00
Client James Lawrence has been scheduled for 2024-06-04_evening
{'2024-06-04_evening': [[<Algorithms.client.Client object at 0x00000143EB43D750>], (['Mierlo', 'Gemert Handel', 'Mierlo'], 176)]}
names: Charles Northrup, location: Deurne Vlierden, availability: ['2024-06-07_evening', '2024-06-12_morning'], appointment_time: 2024-06-02 12:17:00
Client Charles Northrup has been scheduled for 2024-06-07_evening
{'2024-06-04_evening': [[<Algorithms.client.Client object at 0x00000143EB43D750>], (['Mierlo', 'Gemert Handel', 'Mierlo'], 176)], '2024-06-07_evening': [[<Algorithms.client.Client object at 0x00000143CAFBBF70>], (['Mierlo', 'Deurne Vlierden', 'Mierlo'], 158)]}
names: Laurie Osborne, location: Helmond Brandevoort, availability: ['2024-06-07_morning', '2024-06-14_evening'], appointment_time: 2024-06-04 18:56:00
Client

# test offline case

In [51]:
def offline_use_case(config):
    scheduled_definitive_appointments = {}
    timeslot_usage = {}
    
    clients = generate_clients(config['start_date'], config['end_date'], config['min_slots'], config['max_slots'], config['locations'], config['max_timeslot_picks'], timeslot_usage, num_clients=config['num_clients'])
    for client in clients:
        message, scheduled_definitive_appointments = get_definitive_timeslot_clarke(client, scheduled_definitive_appointments, config['distance_matrix_path'])
        print(message)
        print(scheduled_definitive_appointments)

In [52]:
print("\nOffline Use Case Results:")
offline_use_case(config)


Offline Use Case Results:
Client Lillian Hankins has been scheduled for 2024-06-14_evening
{'2024-06-14_evening': [[<Algorithms.client.Client object at 0x00000143EB93BFA0>], (['Mierlo', 'Mierlo', 'Mierlo'], 120)]}
Client Donald Almonte has been scheduled for 2024-06-14_evening with the minimal delta cost.
{'2024-06-14_evening': [[<Algorithms.client.Client object at 0x00000143EB93BFA0>, <Algorithms.client.Client object at 0x00000143EB2E15A0>], (['Mierlo', 'Mierlo', 'Geldrop', 'Mierlo'], 198)]}
Client Maria Black has been scheduled for 2024-06-05_morning
{'2024-06-14_evening': [[<Algorithms.client.Client object at 0x00000143EB93BFA0>, <Algorithms.client.Client object at 0x00000143EB2E15A0>], (['Mierlo', 'Mierlo', 'Geldrop', 'Mierlo'], 198)], '2024-06-05_morning': [[<Algorithms.client.Client object at 0x00000143EB2E2320>], (['Mierlo', 'Someren', 'Mierlo'], 154)]}
Client Heidi Rosales has been scheduled for 2024-06-09_morning
{'2024-06-14_evening': [[<Algorithms.client.Client object at 0x